# Grafový DBMS Neo4j

##  Co se v této části naučíte

* základní pojmy grafově orientovaných databázových systémů
* instalace databáze Neo4j
* dotazovací jazyk Cypher - vkládání, výmaz a modifikace dat


## Grafově orientované DBMS

Grafové DBM se řadí do světa NoSQL databází, mají však některé specifické rysy:

1. nezaměřují se na distrubuované zpracování (i když je podporován v rámci komerční licence)
2. datová representace není orientovaná na tabulky, ale na relace 
3. dotazy využívají optimalizovanou implementaci základních grafových algoritmů (nikoliv tedy obdobu map-reduce)



### Základní entity grafově orientovaných databází

####  Uzel 

representace reálného objekt (instance).

* každý uzel může mít **vlastnosti** (tvořené klíčem a hodnotou hodnota) z nichž nejdůležitější je jméno (identifikátor) uzlu
  * klíč vlastností: řetězec
  * hodnota vlastnosti: libovolný elementární typ (včetně typů pro respresentaci časových údahů) nebo seznam/slovník elementárních hodnot
* každý uzel může mít libovolný (i nulový) počet **popisků** (tagů/labelů), které určují kategorii utlu
* popisy uzlu pouze usnadňuje klasifikaci, nevynucují schéma vlastností (i když jistou závislost nabídky vlastností na kategorii uzlu lze přirozeně očekávat)

#### Vztah

orientované spojení mezi uzly (orientovaná hrana)

* každý vztah může mít **vlastnosti** (podobné vlastnostem uzlů)
* každý vztah má právě jeden **typ** (třída hrany), který učuje význam vztah (mezi uzly může existovat jen jeden vztah určitého typu)
* vztahy jsou orientované tj. každý vztah má počáteční i koncový uzel, orientaci vztahů však lze ignorovat (algoritmy jsou stejně efektivní bez ohledu na orientaci vztahů)

## Neo4j

Neo4j je nejpoužívanější grafově orientovanou databází a je vyvíjena firmou Neo4j ve dvou verzích. Komunitní, která je open source a komerční. Komunitní verzi chybí podpora distribuovaného běhu, ale jinak se jjeí funkčnost neliší.

## Instalace a použití

Pro testovací použití je možno využít oficální docker obraz komunitní verze, jenž by měl být spuštěn následujícím příkazem 
(mapování adresářů lze samozřejmě změnit, změnit lze i uživatele a heslo použité k autentifikaci)
    
```bash
docker run --name testneo4j  \ 
    -p 7474:7474 -p 7687:7687  \  
    -d \
    -v $HOME/neo4j/data:/data \    
    -v $HOME/neo4j/logs:/logs  \  
    -v $HOME/neo4j/import:/var/lib/neo4j/import \
    -v $HOME/neo4j/plugins:/plugins \
    --env NEO4J_AUTH=neo4j/test \
    neo4j:latest
```
Na portu 7414 běží prohlížeč (browser), jenž (mimo) jiné umožňuje interaktivní dotazování databáze. 
Port 7687 je standardní port, na němž běží binární protokol `Bolt` pro připojení k databázi.

 **Úkol**: Nainstalujte a spusťte `neo4j:latest` obraz a pomocí prohlížeče se přihlaste se k databázi pomocí browseru.

**Úkol**: Nainstalujte pythonský modul `neo4j-driver` a spusťte následující program.

```python
from neo4j import GraphDatabase


def create_and_return_greeting(tx, message):
    result = tx.run("CREATE (a:Greeting) "
                    "SET a.message = $message "
                    "RETURN a.message + ', from node ' + id(a)", message=message)
    return result.single()[0]


driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "test"))
with driver.session() as session:
    greeting = session.write_transaction(create_and_return_greeting, "Hello graph")
    print(greeting)
```

Pak uzel (uzly, pokud jste program spustili vicenásobně) smažte v browseru příkazem 
```cypher
MATCH (n:Greeting) DELETE n
```

## Cypher

Cypher je dotazovací jazyk pro DBMS Neo4J (a některé další). Mezi jeho základní charakteristiky patří:

* je deklarativní (jádrem je specifikace vzorů, které určují, co chceme, ne jak toho dosáhnout)
* vychází částečně z SQL
* pro zápis uzlů a vztahů využívá ASCII art

### Základní prvky vzorů

**Uzly** (v kulatých závorkách):
    
`()` = libovolný uzel

`(:Label)` = uzel mající popisek `Label`

`(:Label {property: value})` = uzel mající popisek `Label` a vlastnost `property` s hodnotou `value`

zápis uzlu je často spojen s navázáním uzlu (resp. více uzlů) na proměnnou (lokální v dotazu):

`(n)` = libovolné uzly jsou navázány na proměnou `n`

`(n:Label)` = uzly mající popisek `Label`jsou navázány na proměnou `n`

`(:Label {property: value})` = uzly mající popisek `Label` a vlastnost `property` s hodnotou `value` jsou navázány na proměnnou `n`

**Vztahy** (orientovaná šipka)

`-->` = libovolný orientovaný vztah

`-[:TYPE]->` orientovaný vztah s typem `TYPE`

`-[:TYPE {property: value}]->`  orientovaný vztah s typem `TYPE` mající vlastnost `property` s hodnotou `value`

šipka v zápisu vztahu může být i obráceně orientovaná např. `<-[:TYPE]-` nebo neorientovaná např. `-[:TYPE]-` (popisuje oba směry)

I v případě vztahů lze uvést proměnnou na níž se vztah (či přesněji obecně více vztahů) naváže (níže je to proměnná `r`).

`-[r]->`, `-[r:TYPE]->`, `-[r:TYPE {property: value}]->`

> **Poznámka**: Jazyk Cypher rozpoznává velikosti písmen ale nevynucuje si (jen pro přehlednost, jednotlivé typy identifikátorů jsou v oddělených jmenných prostorech). Jistý úzus ale existuje:
> * proměnné: malá písmena
> * popisky uzlů: první velké
> * typy vztahl: velká písmena


### Vytváření uzlů a vztahů

Pro vytváření uzlů a vztahů slouží příkaz `CREATE`. Nejjednodušší je vytváření izolovaných uzlů (tj. dočasně bez vztahů).

`CREATE (:Label properties)`

v našem případě budeme vytvářet stručný rodokmen Karla IV. jeho předků a potomků. Vytvoření centrálních osob rodokmenu zajistí následující příkazy:

```cypher
CREATE (:Male {name: 'Karel IV.', id: 1});
CREATE (:Male {name: 'Jan Lucemburský', id: 2});
CREATE (:Female {name: 'Eliška Přemyslovna', id: 3});
```

Popiskem je pohlaví osoby a vlastnosti nesou jméno a jednoznačný identifikátor (při vytváření vztahů je nutno osoby jednoznačně identifikovat).

Kromě osob bude v grafu ještě jedna kategorii uzlů: území (ovládané členy rodokmenu).

```cypher
CREATE (:Territory {name: 'Čechy'});
```

Při vytváření vztahů je nutné jednoznačně identifikovat uzly, které vztah spojuje. Proto je nutné využívat sekci MATCH, která najde příslušné uzly a naváže je na proměnnou, až pak je možno vytvářet vztah (hranu).

Následující příkazy vytváří vztah mezi Karlem IV. jeho rodiči a jeho územím.

```cypher
MATCH (p {id: 1}), (f {id: 2}) CREATE (p) -[:FATHER]-> (f);
MATCH (p {id: 1}), (m {id: 3}) CREATE (p) -[:MOTHER]-> (m);
MATCH (r:Male {id: 1}), (t {name: 'Čechy'}) CREATE (r) -[:RULES]-> (t);
```

#### Komplexnější vytváření uzlů a vztahů

Jednotlivé vytváření uzlů a hran není jedinou možností.

Za prvé lze vytvářet celé cesty (tj. uzly a mezi nimi ležící uzly).

Například další předky v přímé mužské linii Vladislava II přidáme příkazem:

```cypher
MATCH (p {name:'Vladislav II'}) CREATE (p) -[:FATHER]-> (:Male {name:'Vladislav I.', id:200}) -[:FATHER]-> (:Male {name:'Vratislav II.', id:202}) -[:FATHER]-> (:Male {name:'Břetislav I.', id:204})
```

Další možností je přidávat vztahy mezi celou množinou uzlů najednou. V našem případě nově přidaní vládci nemají přiděleno území tj. v tomto případě vždy Čechy. Jádrem řešení je najít příkaz který najde všechny mužské předky v přímé linii Vratislava (tj. rekurzivně otce otců).

```cypher
MATCH (p {name:'Vladislav II'}) -[:FATHER*]-> (m), (t {name: 'Čechy'}) CREATE (m) -[:RULES]-> (t)
```

V sekci `MATCH` se najdou všechny uzly `m`, které jsou propojeny s uzlem Vladislava II. libovolně dlouhou posloupností vztahů s typem `FATHER` (všimněte si repetitivní hvězdičky za jménem vztahu). Pak jsou všechny ty tři uzly (označené `m` propojeny vztahem `RULES` s Čechami). 

Lze používat i více sekcí `CREATE`, které se mohou navíc odkazovat na již vložené uzly.  Jediným příkazem tak přidáme Václava Lucemburského bratra Karla IV a vládce Lucemburska.

```cypher
MATCH (f {name:'Jan Lucemburský'}), (m {name: 'Eliška Přemyslovna'}), (t {name: 'Lucembursko'}) 
         CREATE (v:Male {name: 'Václav Lucemburský', id: 300}) // vytváří uzel osoby
         CREATE (v)-[:FATHER]->(f)  // přidává odkaz na jeho otce
         CREATE (v)-[:MOTHER]->(m)  // matku
         CREATE (v)-[:RULES]->(t)   // a území
```

**Úkol**: Vykonejte v prohlížeči následující příkazy vytvářející uzly a vztahy. Budou používány v následujících příkazech.

```cypher
CREATE (:Male {name: 'Karel IV.', id: 1});
CREATE (:Male {name: 'Jan Lucemburský', id: 2});
CREATE (:Female {name: 'Eliška Přemyslovna', id: 3});
CREATE (:Male {name: 'Jindřich VII. Lucemburský', id: 4});
CREATE (:Female {name: 'Markéta Brabantská', id: 5});
CREATE (:Male {name: 'Václav II.', id: 6});
CREATE (:Female {name: 'Guta Habsburská', id: 7});
CREATE (:Male {name: 'Jindřich VI. Lucemburský', id: 8});
CREATE (:Female {name: 'Beatrix z Avesnes', id: 9});
CREATE (:Male {name: 'Jan I. Brabantský', id: 10});
CREATE (:Female {name: 'Markéta z Dampierre', id: 11});
CREATE (:Male {name: 'Přemysl Otakar II.', id: 12});
CREATE (:Female {name: 'Kunhuta Uherská]]', id: 13});
CREATE (:Male {name: 'Rudolf I. Habsburský', id: 14});
CREATE (:Female {name: 'Gertruda z Hohenbergu', id: 15});
CREATE (:Male {name: 'Jindřich V. Lucemburský', id: 16});
CREATE (:Female {name: 'Markéta z Baru', id: 17});
CREATE (:Male {name: 'Balduin z Avesnes', id: 18});
CREATE (:Female {name: 'Felicita z Coucy', id: 19});
CREATE (:Male {name: 'Jindřich III. Brabantský', id: 20});
CREATE (:Female {name: 'Adéla Burgundská', id: 21});
CREATE (:Male {name: 'Vít z Dampierre', id: 22});
CREATE (:Female {name: 'Matylda z Béthuné', id: 23});
CREATE (:Male {name: 'Václav I.', id: 24});
CREATE (:Female {name: 'Kunhuta Štaufská', id: 25});
CREATE (:Male {name: 'Rostislav Haličský', id: 26});
CREATE (:Female {name: 'Anna Uherská', id: 27});
CREATE (:Male {name: 'Albrecht IV. Habsburský', id: 28});
CREATE (:Female {name: 'Hedvika z Kyburgu', id: 29});
CREATE (:Male {name: 'Burkhard III. z Hohenbergu', id: 30});
CREATE (:Female {name: 'Mechtilda z Tübingenu', id: 31});
CREATE (:Male {name: 'Walram III. Limburský', id: 32});
CREATE (:Female {name: 'Ermesinda Lucemburská', id: 33});
CREATE (:Male {name: 'Přemysl Otakar I.', id: 48});
CREATE (:Female {name: 'Konstancie Uherská', id: 49});
CREATE (:Male {name: 'Vladislav II', id: 96});
CREATE (:Female {name: 'Judita Durynská', id: 97});
CREATE (:Female {name: 'Blanka z Valois', id: 101});
CREATE (:Female {name: 'Markéta Lucemburská', id: 103});
CREATE (:Female {name: 'Kateřina Lucemburská', id: 105});
CREATE (:Female {name: 'Anna Svídnická', id: 111});
CREATE (:Female {name: 'Alžběta Lucemburská', id: 113});
CREATE (:Male {name: 'Václav IV.', id: 114});
CREATE (:Female {name: 'Alžbeta Pomořanská', id: 121});
CREATE (:Female {name: 'Anna Lucemburská', id: 123});
CREATE (:Male {name: 'Zikmud Lucemburský', id: 124});
CREATE (:Male {name: 'Jan Zhořelecký', id: 126});
CREATE (:Female {name: 'Markéta Lucemburská', id: 127});
MATCH (p {id: 1}), (f {id: 2}) CREATE (p) -[:FATHER]-> (f);
MATCH (p {id: 2}), (f {id: 4}) CREATE (p) -[:FATHER]-> (f);
MATCH (p {id: 3}), (f {id: 6}) CREATE (p) -[:FATHER]-> (f);
MATCH (p {id: 4}), (f {id: 8}) CREATE (p) -[:FATHER]-> (f);
MATCH (p {id: 5}), (f {id: 10}) CREATE (p) -[:FATHER]-> (f);
MATCH (p {id: 6}), (f {id: 12}) CREATE (p) -[:FATHER]-> (f);
MATCH (p {id: 7}), (f {id: 14}) CREATE (p) -[:FATHER]-> (f);
MATCH (p {id: 8}), (f {id: 16}) CREATE (p) -[:FATHER]-> (f);
MATCH (p {id: 9}), (f {id: 18}) CREATE (p) -[:FATHER]-> (f);
MATCH (p {id: 10}), (f {id: 20}) CREATE (p) -[:FATHER]-> (f);
MATCH (p {id: 11}), (f {id: 22}) CREATE (p) -[:FATHER]-> (f);
MATCH (p {id: 12}), (f {id: 24}) CREATE (p) -[:FATHER]-> (f);
MATCH (p {id: 13}), (f {id: 26}) CREATE (p) -[:FATHER]-> (f);
MATCH (p {id: 14}), (f {id: 28}) CREATE (p) -[:FATHER]-> (f);
MATCH (p {id: 15}), (f {id: 30}) CREATE (p) -[:FATHER]-> (f);
MATCH (p {id: 16}), (f {id: 32}) CREATE (p) -[:FATHER]-> (f);
MATCH (p {id: 24}), (f {id: 48}) CREATE (p) -[:FATHER]-> (f);
MATCH (p {id: 48}), (f {id: 96}) CREATE (p) -[:FATHER]-> (f);
MATCH (p {id: 103}), (f {id: 1}) CREATE (p) -[:FATHER]-> (f);
MATCH (p {id: 105}), (f {id: 1}) CREATE (p) -[:FATHER]-> (f);
MATCH (p {id: 113}), (f {id: 1}) CREATE (p) -[:FATHER]-> (f);
MATCH (p {id: 114}), (f {id: 1}) CREATE (p) -[:FATHER]-> (f);
MATCH (p {id: 123}), (f {id: 1}) CREATE (p) -[:FATHER]-> (f);
MATCH (p {id: 124}), (f {id: 1}) CREATE (p) -[:FATHER]-> (f);
MATCH (p {id: 126}), (f {id: 1}) CREATE (p) -[:FATHER]-> (f);
MATCH (p {id: 127}), (f {id: 1}) CREATE (p) -[:FATHER]-> (f);
MATCH (p {id: 1}), (m {id: 3}) CREATE (p) -[:MOTHER]-> (m);
MATCH (p {id: 2}), (m {id: 5}) CREATE (p) -[:MOTHER]-> (m);
MATCH (p {id: 3}), (m {id: 7}) CREATE (p) -[:MOTHER]-> (m);
MATCH (p {id: 4}), (m {id: 9}) CREATE (p) -[:MOTHER]-> (m);
MATCH (p {id: 5}), (m {id: 11}) CREATE (p) -[:MOTHER]-> (m);
MATCH (p {id: 6}), (m {id: 13}) CREATE (p) -[:MOTHER]-> (m);
MATCH (p {id: 7}), (m {id: 15}) CREATE (p) -[:MOTHER]-> (m);
MATCH (p {id: 8}), (m {id: 17}) CREATE (p) -[:MOTHER]-> (m);
MATCH (p {id: 9}), (m {id: 19}) CREATE (p) -[:MOTHER]-> (m);
MATCH (p {id: 10}), (m {id: 21}) CREATE (p) -[:MOTHER]-> (m);
MATCH (p {id: 11}), (m {id: 23}) CREATE (p) -[:MOTHER]-> (m);
MATCH (p {id: 12}), (m {id: 25}) CREATE (p) -[:MOTHER]-> (m);
MATCH (p {id: 13}), (m {id: 27}) CREATE (p) -[:MOTHER]-> (m);
MATCH (p {id: 14}), (m {id: 29}) CREATE (p) -[:MOTHER]-> (m);
MATCH (p {id: 15}), (m {id: 31}) CREATE (p) -[:MOTHER]-> (m);
MATCH (p {id: 16}), (m {id: 33}) CREATE (p) -[:MOTHER]-> (m);
MATCH (p {id: 24}), (m {id: 49}) CREATE (p) -[:MOTHER]-> (m);
MATCH (p {id: 48}), (m {id: 97}) CREATE (p) -[:MOTHER]-> (m);
MATCH (p {id: 103}), (m {id: 101}) CREATE (p) -[:MOTHER]-> (m);
MATCH (p {id: 105}), (m {id: 101}) CREATE (p) -[:MOTHER]-> (m);
MATCH (p {id: 113}), (m {id: 111}) CREATE (p) -[:MOTHER]-> (m);
MATCH (p {id: 114}), (m {id: 111}) CREATE (p) -[:MOTHER]-> (m);
MATCH (p {id: 123}), (m {id: 121}) CREATE (p) -[:MOTHER]-> (m);
MATCH (p {id: 124}), (m {id: 121}) CREATE (p) -[:MOTHER]-> (m);
MATCH (p {id: 126}), (m {id: 121}) CREATE (p) -[:MOTHER]-> (m);
MATCH (p {id: 127}), (m {id: 121}) CREATE (p) -[:MOTHER]-> (m);
CREATE (:Territory {name: 'Čechy'});
MATCH (r:Male {id: 1}), (t {name: 'Čechy'}) CREATE (r) -[:RULES]-> (t);
MATCH (r:Male {id: 2}), (t {name: 'Čechy'}) CREATE (r) -[:RULES]-> (t);
MATCH (r:Male {id: 6}), (t {name: 'Čechy'}) CREATE (r) -[:RULES]-> (t);
MATCH (r:Male {id: 12}), (t {name: 'Čechy'}) CREATE (r) -[:RULES]-> (t);
MATCH (r:Male {id: 24}), (t {name: 'Čechy'}) CREATE (r) -[:RULES]-> (t);
MATCH (r:Male {id: 48}), (t {name: 'Čechy'}) CREATE (r) -[:RULES]-> (t);
MATCH (r:Male {id: 96}), (t {name: 'Čechy'}) CREATE (r) -[:RULES]-> (t);
MATCH (r:Male {id: 114}), (t {name: 'Čechy'}) CREATE (r) -[:RULES]-> (t);
MATCH (r:Male {id: 124}), (t {name: 'Čechy'}) CREATE (r) -[:RULES]-> (t);
CREATE (:Territory {name: 'Lucembursko'});
MATCH (r:Male {id: 1}), (t {name: 'Lucembursko'}) CREATE (r) -[:RULES]-> (t);
MATCH (r:Male {id: 2}), (t {name: 'Lucembursko'}) CREATE (r) -[:RULES]-> (t);
MATCH (r:Male {id: 4}), (t {name: 'Lucembursko'}) CREATE (r) -[:RULES]-> (t);
MATCH (r:Male {id: 8}), (t {name: 'Lucembursko'}) CREATE (r) -[:RULES]-> (t);
MATCH (r:Male {id: 16}), (t {name: 'Lucembursko'}) CREATE (r) -[:RULES]-> (t);
MATCH (r:Male {id: 32}), (t {name: 'Lucembursko'}) CREATE (r) -[:RULES]-> (t);
CREATE (:Territory {name: 'SŘŘ'});
MATCH (r:Male {id: 1}), (t {name: 'SŘŘ'}) CREATE (r) -[:RULES]-> (t);
MATCH (r:Male {id: 4}), (t {name: 'SŘŘ'}) CREATE (r) -[:RULES]-> (t);
MATCH (r:Male {id: 14}), (t {name: 'SŘŘ'}) CREATE (r) -[:RULES]-> (t);
MATCH (r:Male {id: 114}), (t {name: 'SŘŘ'}) CREATE (r) -[:RULES]-> (t);
MATCH (r:Male {id: 124}), (t {name: 'SŘŘ'}) CREATE (r) -[:RULES]-> (t);
CREATE (:Territory {name: 'Brabantsko'});
MATCH (r:Male {id: 10}), (t {name: 'Brabantsko'}) CREATE (r) -[:RULES]-> (t);
MATCH (r:Male {id: 20}), (t {name: 'Brabantsko'}) CREATE (r) -[:RULES]-> (t);
CREATE (:Territory {name: 'Limburg'});
MATCH (r:Male {id: 10}), (t {name: 'Limburg'}) CREATE (r) -[:RULES]-> (t);
MATCH (r:Male {id: 32}), (t {name: 'Limburg'}) CREATE (r) -[:RULES]-> (t);
CREATE (:Territory {name: 'Rakousko'});
MATCH (r:Male {id: 12}), (t {name: 'Rakousko'}) CREATE (r) -[:RULES]-> (t);
MATCH (r:Male {id: 14}), (t {name: 'Rakousko'}) CREATE (r) -[:RULES]-> (t);
CREATE (:Territory {name: 'Lotrinsko'});
MATCH (r:Male {id: 20}), (t {name: 'Lotrinsko'}) CREATE (r) -[:RULES]-> (t);
CREATE (:Territory {name: 'Flandry'});
MATCH (r:Male {id: 22}), (t {name: 'Flandry'}) CREATE (r) -[:RULES]-> (t);
CREATE (:Territory {name: 'Namur'});
MATCH (r:Male {id: 22}), (t {name: 'Namur'}) CREATE (r) -[:RULES]-> (t);
CREATE (:Territory {name: 'Slavonsko'});
MATCH (r:Male {id: 26}), (t {name: 'Slavonsko'}) CREATE (r) -[:RULES]-> (t);
CREATE (:Territory {name: 'Aargau'});
MATCH (r:Male {id: 28}), (t {name: 'Aargau'}) CREATE (r) -[:RULES]-> (t);
CREATE (:Territory {name: 'Horní Alsasko'});
MATCH (r:Male {id: 28}), (t {name: 'Horní Alsasko'}) CREATE (r) -[:RULES]-> (t);
CREATE (:Territory {name: 'Uhry'});
MATCH (r:Male {id: 124}), (t {name: 'Uhry'}) CREATE (r) -[:RULES]-> (t);
CREATE (:Territory {name: 'Braniborsko'});
MATCH (r:Male {id: 126}), (t {name: 'Braniborsko'}) CREATE (r) -[:RULES]-> (t);
                                                                            
MATCH (p {name:'Vladislav II'}) CREATE (p) -[:FATHER]-> (:Male {name:'Vladislav I.', id:200}) -[:FATHER]-> (:Male {name:'Vratislav II.', id:202}) -[:FATHER]-> (:Male {name:'Břetislav I.', id:204});

MATCH (p {name:'Vladislav II'}) -[:FATHER*]-> (m), (t {name: 'Čechy'}) CREATE (m) -[:RULES]-> (t);
                                                                                               
MATCH (f {name:'Jan Lucemburský'}), (m {name: 'Eliška Přemyslovna'}), (t {name: 'Lucembursko'}) 
         CREATE (v:Male {name: 'Václav Lucemburský', id: 300}) // vytváří uzel osoby
         CREATE (v)-[:FATHER]->(f)  // přidává odkaz na jeho otce
         CREATE (v)-[:MOTHER]->(m)  // matku
         CREATE (v)-[:RULES]->(t)   // a území                                                                                               
```

**Úkol**: Navrhněte odpovídající databázi v SQL, včetně uložené procedury pro vkládání ve stylu posledního čtyřdílného CREATE (Václav Lucemburský).

Pokud jsou data o uzlech a vztazích uložena v CSV souborech lze využít příkazu `LOAD CSV`. Tento příkaz má dvě části, v prvním se specifikuje CSV soubor (umístění, formát), druhým je standardní `CREATE` jenž vytváří uzly resp. vztahy a to s využitím dat z jednotlivých sloupců CSV (část CREATE je provedena pro každý řádek CSV souboru).

popis příkazu s příklady využití viz: https://neo4j.com/docs/cypher-manual/current/clauses/load-csv/

> **Úkol**: Jak Cypher řeší případy, kdy řádky obsahují odkazy na data, která teprve následují (tj. např. řádek s údaji o osobě odkazuje na osobu na následujícím řádku)


### Výmaz 

Příkaz výmazu se je obdobou příkazu CREATE. Kromě záměny klíčového slova na DELETE jsou zde však dva základní rozdíly:

* v sekci DELETE se většinou používá jen proměnná či seznam proměnných
* nejdříve je nutné vymazat vztahy až pak uzly (tj. právě naopak oproti vytváření)

V naší ukázkové databázi tak nemůžeme přímo smazat žádný uzel, neboť všechny jsou zapojeny do vztahů. 

Chceme-li například smazat zemi Rakousko musíme nejdříve najít a smazat vztahy do níž je zapojena.

Nejdříve tyto vztahy najdeme:

```cypher
MATCH ()-[r:RULES]->(:Territory {name:"Rakousko"}) RETURN r
```

Zjistíme, že existují dva takové vztahy (Přemysl Otakar II a Rudolf I). Vymažeme je tak, že zaměníme klíčové slovo RETURN za DELETE.

Pak teprve můžeme v smazat příslušný uzel:

```cypher
MATCH (:Territory {name:"Rakousko"}) RETURN r
```

Dvoufázový výmaz lze v novějších verzích Neo4j nahradit příkazem `DETACH DELETE`, který kromě uzlů maže i všechny incidující vztahy.

Výmaz všech zemí (včetně informací, kdo jim vládne):

```cypher
MATCH (t:Territory) DETACH DELETE t
```

> **Otázka**: Jakým příkazem lze smazat celou databázi.

### Práce s vlastnostmi

Pro práci s jednotlivými vlastnostmi slouží příkazy `SET` (přidání nebo změna vlastnosti) nebo `REMOVE`(odstraněnní vlastnosti).

popisy těchto příkazů s popisy lze nalézt: 

> **Úkol**: Prozkoumejte typový systém vlastností. Zaměřte se na typy 